Data Cleaning Notebook

In [1]:
import pandas as pd
import numpy as np

# Display full tables
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [2]:
df = pd.read_csv("../data/raw_dataset.csv")

before_shape = df.shape
print("Initial shape:", before_shape)
df.head()

Initial shape: (891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
print("DataFrame Info:")
df.info()

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [4]:
print("Missing values per column (full list):")
for col in df.columns:
    missing_count = df[col].isnull().sum()
    missing_pct = (missing_count / df.shape[0] * 100).round(2)
    print(f"{col}: {missing_count} missing ({missing_pct}%)")

Missing values per column (full list):
PassengerId: 0 missing (0.0%)
Survived: 0 missing (0.0%)
Pclass: 0 missing (0.0%)
Name: 0 missing (0.0%)
Sex: 0 missing (0.0%)
Age: 177 missing (19.87%)
SibSp: 0 missing (0.0%)
Parch: 0 missing (0.0%)
Ticket: 0 missing (0.0%)
Fare: 0 missing (0.0%)
Cabin: 687 missing (77.1%)
Embarked: 2 missing (0.22%)


In [5]:
duplicates = df.duplicated().sum()
print("Duplicate rows count:", duplicates)

Duplicate rows count: 0


In [6]:
print("Summary statistics (numeric + object):")
display(df.describe(include='all'))

Summary statistics (numeric + object):


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,891.000000,891.000000,891.000000,891,891,714.000000,891.000000,891.000000,891,891.000000,204,889
unique,NaN,NaN,NaN,891,2,NaN,NaN,NaN,681,NaN,147,3
top,NaN,NaN,NaN,"Braund, Mr. Owen Harris",male,NaN,NaN,NaN,347082,NaN,G6,S
freq,NaN,NaN,NaN,1,577,NaN,NaN,NaN,7,NaN,4,644
mean,446.000000,0.383838,2.308642,NaN,NaN,29.699118,0.523008,0.381594,NaN,32.204208,NaN,NaN
std,257.353842,0.486592,0.836071,NaN,NaN,14.526497,1.102743,0.806057,NaN,49.693429,NaN,NaN
min,1.000000,0.000000,1.000000,NaN,NaN,0.420000,0.000000,0.000000,NaN,0.000000,NaN,NaN
25%,223.500000,0.000000,2.000000,NaN,NaN,20.125000,0.000000,0.000000,NaN,7.910400,NaN,NaN
50%,446.000000,0.000000,3.000000,NaN,NaN,28.000000,0.000000,0.000000,NaN,14.454200,NaN,NaN
75%,668.500000,1.000000,3.000000,NaN,NaN,38.000000,1.000000,0.000000,NaN,31.000000,NaN,NaN


In [7]:
df.replace(['?', ''], np.nan, inplace=True)
print("Normalized '?' and empty strings to NaN (if present).")

Normalized '?' and empty strings to NaN (if present).


In [8]:
# Fill numeric columns
if 'Age' in df.columns:
    df['Age'] = df['Age'].fillna(df['Age'].median())

# Fill categorical columns
if 'Embarked' in df.columns:
    mode_value = df['Embarked'].mode()[0]
    df['Embarked'] = df['Embarked'].fillna(mode_value)

# Drop columns with >50% missing values
if 'Cabin' in df.columns and df['Cabin'].isnull().mean() > 0.5:
    df = df.drop(columns=['Cabin'])
    print("Dropped 'Cabin' column (too many missing values).")

print("Remaining missing values per column:")
for col in df.columns:
    print(f"{col}: {df[col].isnull().sum()} missing")

Dropped 'Cabin' column (too many missing values).
Remaining missing values per column:
PassengerId: 0 missing
Survived: 0 missing
Pclass: 0 missing
Name: 0 missing
Sex: 0 missing
Age: 0 missing
SibSp: 0 missing
Parch: 0 missing
Ticket: 0 missing
Fare: 0 missing
Embarked: 0 missing


In [9]:
pre_rows = df.shape[0]
df = df.drop_duplicates()
print("Duplicates removed:", pre_rows - df.shape[0])

# Strip whitespace and normalize strings
for c in df.select_dtypes(include=['object']).columns:
    df[c] = df[c].astype(str).str.strip()

if 'Sex' in df.columns:
    df['Sex'] = df['Sex'].str.lower()
if 'Embarked' in df.columns:
    df['Embarked'] = df['Embarked'].str.upper()

Duplicates removed: 0


In [10]:
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()

for col in num_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    df[col] = df[col].clip(lower=lower, upper=upper)

print("Applied IQR clipping to numeric columns.")

Applied IQR clipping to numeric columns.


In [11]:
print("Rows before cleaning:", before_shape[0])
print("Rows after cleaning:", df.shape[0])

df.to_csv("../data/cleaned_dataset.csv", index=False)
print("Saved cleaned dataset to ../data/cleaned_dataset.csv")

Rows before cleaning: 891
Rows after cleaning: 891
Saved cleaned dataset to ../data/cleaned_dataset.csv
